In [1]:
import json
import os
import time
import random
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import shutil

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms
from torchvision import models
from torchsummary import summary
from sklearn.model_selection import train_test_split

from model_pytorch import EfficientNet
from utils import Bar,Logger, AverageMeter, accuracy, mkdir_p, savefig
from warmup_scheduler import GradualWarmupScheduler

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import collections

In [2]:
# GPU Device
gpu_id = 0
os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu_id)
use_cuda = torch.cuda.is_available()
print("GPU device %d:" %(gpu_id), use_cuda)

GPU device 0: True


# Arguments

In [3]:
data_dir = '/media/data2/dataset/GAN_ImageData/PGGAN_128'

In [4]:
pretrained = ''
resume = ''

In [5]:
# Model
model_name = 'efficientnet-b1' # b0-b7 scale

# Optimization
num_classes = 2
epochs = 300
start_epoch = 0
train_batch = 200
test_batch = 200
lr = 0.1
schedule = [20, 75, 175, 250]
momentum = 0.9
gamma = 0.1 # LR is multiplied by gamma on schedule

# CheckPoint
checkpoint = './log/pggan/128/b0' # dir
if not os.path.isdir(checkpoint):
    os.mkdir(checkpoint)
num_workers = 8

# Seed
manual_seed = 7
random.seed(manual_seed)
torch.cuda.manual_seed_all(manual_seed)

# Image
size = (128, 128)

best_acc = 0

In [6]:
state = {}
state['num_classes'] = num_classes
state['epochs'] = epochs
state['start_epoch'] = start_epoch
state['train_batch'] = train_batch
state['test_batch'] = test_batch
state['lr'] = lr
state['schedule'] = schedule
state['momentum'] = momentum
state['gamma'] = gamma

# Dataset

In [7]:
train_dir = os.path.join(data_dir, 'train')
val_dir = os.path.join(data_dir, 'validation')    
train_aug = transforms.Compose([
    transforms.RandomAffine(degrees=2, translate=(0.02, 0.02), scale=(0.98, 1.02), shear=2, fillcolor=(124,117,104)),
    transforms.Resize(size),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.RandomErasing(p=0.3, scale=(0.02, 0.10), ratio=(0.3, 3.3), value=0, inplace=True),
])
val_aug = transforms.Compose([
    transforms.Resize(size),
    transforms.ToTensor(),
])

# pin_memory : cuda pin memeory use
train_loader = DataLoader(datasets.ImageFolder(train_dir, transform=train_aug),
                          batch_size=train_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
val_loader = DataLoader(datasets.ImageFolder(val_dir, val_aug),
                       batch_size=test_batch, shuffle=True, num_workers=num_workers, pin_memory=True)

# Model

In [8]:
model = EfficientNet.from_name(model_name, num_classes=num_classes,
                              override_params={'dropout_rate':0.2, 'drop_connect_rate':0.2})

# Pre-trained
if pretrained:
    print("=> using pre-trained model '{}'".format(pretrained))
    model.load_state_dict(torch.load(pretrained)['state_dict'])

In [9]:
model.to('cuda')
cudnn.benchmark = True
print('    Total params: %.2fM' % (sum(p.numel() for p in model.parameters())/1000000.0))

    Total params: 6.52M


# Loss

In [10]:
criterion = nn.CrossEntropyLoss().cuda()
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=1e-4)
scheduler_cosine = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)
scheduler_warmup = GradualWarmupScheduler(optimizer, multiplier=8, total_epoch=10, after_scheduler=scheduler_cosine)

In [11]:
# Resume
if resume:
    print('==> Resuming from checkpoint..')
    checkpoint = os.path.dirname(resume)
#     checkpoint = torch.load(resume)
    resume = torch.load(resume)
    best_acc = resume['best_acc']
    start_epoch = resume['epoch']
    model.load_state_dict(resume['state_dict'])
    optimizer.load_state_dict(resume['optimizer'])
    logger = Logger(os.path.join(checkpoint, 'log.txt'), resume=True)
else:
    logger = Logger(os.path.join(checkpoint, 'log.txt'))
    logger.set_names(['Learning Rate', 'Train Loss', 'Valid Loss', 'Train Acc.', 'Valid Acc.'])

# Train

In [12]:
def train(train_loader, model, criterion, optimizer, epoch, use_cuda):
    model.train()
    torch.set_grad_enabled(True)
    
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    end = time.time()
    
    bar = Bar('Processing', max=len(train_loader))
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        batch_size = inputs.size(0)
        if batch_size < train_batch:
            continue
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        inputs, targets = torch.autograd.Variable(inputs), torch.autograd.Variable(targets)

        # compute output
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # measure accuracy and record loss
        prec1 = accuracy(outputs.data, targets.data)
        losses.update(loss.data.tolist(), inputs.size(0))
        top1.update(prec1[0], inputs.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        bar.suffix  = '({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | Total: {total:} | ETA: {eta:} | Loss: {loss:.4f} | top1: {top1: .4f} '.format(
                    batch=batch_idx + 1,
                    size=len(train_loader),
                    data=data_time.val,
                    bt=batch_time.val,
                    total=bar.elapsed_td,
                    eta=bar.eta_td,
                    loss=losses.avg,
                    top1=top1.avg,
                    )
        bar.next()
        if batch_idx % 10 == 0:
            print('{batch}/{size} Data:{data:.3f} | Batch:{bt:.3f} | Total:{total:} | ETA:{eta:} | Loss:{loss:} | top1:{tp1:}'.format(
                 batch=batch_idx+1, size=len(train_loader), data=data_time.val, bt=batch_time.val, total=bar.elapsed_td, eta=bar.eta_td, loss=losses.avg, tp1=top1.avg))
    bar.finish()
    return (losses.avg, top1.avg)

In [13]:
def test(val_loader, model, criterion, epoch, use_cuda):
    global best_acc

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to evaluate mode
    model.eval()
    torch.set_grad_enabled(False)

    end = time.time()
    bar = Bar('Processing', max=len(val_loader))
    for batch_idx, (inputs, targets) in enumerate(val_loader):
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        inputs, targets = torch.autograd.Variable(inputs, volatile=True), torch.autograd.Variable(targets)

        # compute output
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # measure accuracy and record loss
        prec1 = accuracy(outputs.data, targets.data)
        losses.update(loss.data.tolist(), inputs.size(0))
        top1.update(prec1[0], inputs.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        bar.suffix  = '({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | Total: {total:} | ETA: {eta:} | Loss: {loss:} | top1: {top1:}'.format(
                    batch=batch_idx + 1,
                    size=len(val_loader),
                    data=data_time.avg,
                    bt=batch_time.avg,
                    total=bar.elapsed_td,
                    eta=bar.eta_td,
                    loss=losses.avg,
                    top1=top1.avg,)
        bar.next()
    print('{batch}/{size} Data:{data:.3f} | Batch:{bt:.3f} | Total:{total:} | ETA:{eta:} | Loss:{loss:} | top1:{tp1:}'.format(
         batch=batch_idx+1, size=len(val_loader), data=data_time.val, bt=batch_time.val, total=bar.elapsed_td, eta=bar.eta_td, loss=losses.avg, tp1=top1.avg))
    bar.finish()
    return (losses.avg, top1.avg)

In [14]:
def save_checkpoint(state, is_best, checkpoint='checkpoint', filename='checkpoint.pth.tar'):
    filepath = os.path.join(checkpoint, filename)
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(checkpoint, 'model_best.pth.tar'))

def adjust_learning_rate(optimizer, epoch):
    global state
    lr_set = [1, 1e-1, 1e-2, 1e-3, 1e-4, 1e-5]
    lr_list = schedule.copy()
    lr_list.append(epoch)
    lr_list.sort()
    idx = lr_list.index(epoch)
    state['lr'] *= lr_set[idx]
    for param_group in optimizer.param_groups:
        param_group['lr'] = state['lr']

In [ ]:
for epoch in range(start_epoch, epochs):
    state['lr'] = optimizer.state_dict()['param_groups'][0]['lr']
    adjust_learning_rate(optimizer, epoch)
    print('\nEpoch: [%d | %d] LR: %f' % (epoch + 1, epochs, state['lr']))
    
    train_loss, train_acc = train(train_loader, model, criterion, optimizer, epoch, use_cuda)
    test_loss, test_acc = test(val_loader, model, criterion, epoch, use_cuda)
    
    logger.append([state['lr'], train_loss, test_loss, train_acc, test_acc])
    scheduler_warmup.step()

    is_best = test_acc > best_acc
    best_acc = max(test_acc, best_acc)
    save_checkpoint({
        'epoch': epoch + 1,
        'state_dict' : model.state_dict(),
        'acc': test_acc,
        'best_acc': best_acc,
        'optimizer': optimizer.state_dict(),
    }, is_best, checkpoint=checkpoint)


Epoch: [1 | 300] LR: 0.100000
1/643 Data:0.995 | Batch:3.991 | Total:0:00:03 | ETA:0:42:43 | Loss:0.6953132152557373 | top1:52.0
11/643 Data:0.001 | Batch:0.328 | Total:0:00:07 | ETA:0:07:23 | Loss:2.6947527581995185 | top1:51.3636360168457
21/643 Data:0.000 | Batch:0.322 | Total:0:00:10 | ETA:0:03:24 | Loss:1.9614282080105372 | top1:50.57142639160156
31/643 Data:0.000 | Batch:0.322 | Total:0:00:13 | ETA:0:03:18 | Loss:1.6019820551718436 | top1:50.709678649902344
41/643 Data:0.000 | Batch:0.323 | Total:0:00:17 | ETA:0:03:15 | Loss:1.3935006478937662 | top1:50.79268264770508
51/643 Data:0.000 | Batch:0.323 | Total:0:00:20 | ETA:0:03:12 | Loss:1.2583507404607885 | top1:50.686275482177734
61/643 Data:0.000 | Batch:0.323 | Total:0:00:23 | ETA:0:03:08 | Loss:1.1696836215550783 | top1:50.66393280029297
71/643 Data:0.001 | Batch:0.324 | Total:0:00:26 | ETA:0:03:05 | Loss:1.1094274965810105 | top1:50.73239517211914
81/643 Data:0.000 | Batch:0.323 | Total:0:00:29 | ETA:0:03:02 | Loss:1.0622344

/home/cutz/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.


161/161 Data:0.000 | Batch:0.727 | Total:0:00:26 | ETA:0:00:00 | Loss:0.6934156716798325 | top1:50.000003814697266

Epoch: [2 | 300] LR: 0.170000
1/643 Data:2.747 | Batch:3.717 | Total:0:00:03 | ETA:0:39:47 | Loss:0.6970978379249573 | top1:46.5
11/643 Data:0.001 | Batch:0.479 | Total:0:00:08 | ETA:0:08:55 | Loss:0.6967205947095697 | top1:49.772727966308594
21/643 Data:0.005 | Batch:0.539 | Total:0:00:13 | ETA:0:05:03 | Loss:0.6968837380409241 | top1:49.66666793823242
31/643 Data:0.001 | Batch:0.400 | Total:0:00:18 | ETA:0:04:58 | Loss:0.7000131030236522 | top1:49.7580680847168
41/643 Data:0.001 | Batch:0.690 | Total:0:00:23 | ETA:0:05:17 | Loss:0.6993361289908246 | top1:50.0121955871582
51/643 Data:0.001 | Batch:0.421 | Total:0:00:28 | ETA:0:04:56 | Loss:0.6993764486967349 | top1:49.990196228027344
61/643 Data:0.001 | Batch:0.612 | Total:0:00:33 | ETA:0:04:21 | Loss:0.6991090256659711 | top1:50.12295150756836
71/643 Data:0.001 | Batch:0.659 | Total:0:00:38 | ETA:0:05:01 | Loss:0.698953

51/643 Data:0.000 | Batch:0.330 | Total:0:00:19 | ETA:0:03:15 | Loss:0.6982103481012232 | top1:49.431373596191406
61/643 Data:0.001 | Batch:0.386 | Total:0:00:23 | ETA:0:03:34 | Loss:0.697620114342111 | top1:49.1065559387207
71/643 Data:0.000 | Batch:0.329 | Total:0:00:26 | ETA:0:03:17 | Loss:0.6973106592473849 | top1:49.154930114746094
81/643 Data:0.000 | Batch:0.331 | Total:0:00:29 | ETA:0:03:07 | Loss:0.6967450630517653 | top1:49.51234436035156
91/643 Data:0.000 | Batch:0.331 | Total:0:00:33 | ETA:0:03:03 | Loss:0.6967844674875449 | top1:49.43955993652344
101/643 Data:0.000 | Batch:0.329 | Total:0:00:36 | ETA:0:03:00 | Loss:0.6965279738501747 | top1:49.653465270996094
111/643 Data:0.001 | Batch:0.361 | Total:0:00:39 | ETA:0:03:02 | Loss:0.6963077337892206 | top1:49.77928161621094
121/643 Data:0.000 | Batch:0.380 | Total:0:00:43 | ETA:0:03:14 | Loss:0.6962079642232785 | top1:49.79338836669922
131/643 Data:0.001 | Batch:0.378 | Total:0:00:47 | ETA:0:03:07 | Loss:0.6963579313445637 | t

111/643 Data:0.001 | Batch:0.414 | Total:0:00:45 | ETA:0:04:22 | Loss:0.6963475201580975 | top1:50.20270538330078
121/643 Data:0.001 | Batch:0.419 | Total:0:00:49 | ETA:0:03:51 | Loss:0.6961993900212374 | top1:50.1694221496582
131/643 Data:0.001 | Batch:0.337 | Total:0:00:54 | ETA:0:03:41 | Loss:0.6959406974661442 | top1:50.13358688354492
141/643 Data:0.001 | Batch:0.519 | Total:0:00:59 | ETA:0:04:19 | Loss:0.6957267425584455 | top1:50.29433059692383
151/643 Data:0.001 | Batch:0.496 | Total:0:01:03 | ETA:0:03:37 | Loss:0.6957949467052688 | top1:50.298011779785156
161/643 Data:0.001 | Batch:0.351 | Total:0:01:07 | ETA:0:03:31 | Loss:0.6956418389859407 | top1:50.34161376953125
171/643 Data:0.001 | Batch:0.335 | Total:0:01:11 | ETA:0:02:58 | Loss:0.6955022139158863 | top1:50.345027923583984
181/643 Data:0.001 | Batch:0.414 | Total:0:01:15 | ETA:0:02:43 | Loss:0.695345247648039 | top1:50.339778900146484
191/643 Data:0.001 | Batch:0.444 | Total:0:01:19 | ETA:0:03:10 | Loss:0.695220784679133

...

471/643 Data:0.000 | Batch:0.328 | Total:0:02:38 | ETA:0:00:58 | Loss:0.41007597678026575 | top1:80.94267272949219
481/643 Data:0.000 | Batch:0.337 | Total:0:02:41 | ETA:0:00:55 | Loss:0.4100496035355788 | top1:80.93971252441406
491/643 Data:0.000 | Batch:0.328 | Total:0:02:44 | ETA:0:00:51 | Loss:0.40988654517350515 | top1:80.9470443725586
501/643 Data:0.000 | Batch:0.328 | Total:0:02:47 | ETA:0:00:47 | Loss:0.40997245955372047 | top1:80.93912506103516
511/643 Data:0.000 | Batch:0.336 | Total:0:02:51 | ETA:0:00:44 | Loss:0.4102177405893919 | top1:80.92172241210938
521/643 Data:0.001 | Batch:0.336 | Total:0:02:54 | ETA:0:00:41 | Loss:0.4097538518928521 | top1:80.9596939086914
531/643 Data:0.000 | Batch:0.415 | Total:0:02:58 | ETA:0:00:38 | Loss:0.40938373962364627 | top1:80.99152374267578
541/643 Data:0.000 | Batch:0.340 | Total:0:03:01 | ETA:0:00:35 | Loss:0.40943411863004436 | top1:80.98613739013672
551/643 Data:0.000 | Batch:0.335 | Total:0:03:04 | ETA:0:00:31 | Loss:0.4095081865462

531/643 Data:0.001 | Batch:0.605 | Total:0:03:23 | ETA:0:00:59 | Loss:0.40792587893381854 | top1:81.01318359375
541/643 Data:0.001 | Batch:0.568 | Total:0:03:29 | ETA:0:01:01 | Loss:0.407934296263344 | top1:81.00923919677734
551/643 Data:0.001 | Batch:0.588 | Total:0:03:35 | ETA:0:00:55 | Loss:0.4078355107030505 | top1:81.0108871459961
561/643 Data:0.006 | Batch:0.511 | Total:0:03:40 | ETA:0:00:41 | Loss:0.40770884572715893 | top1:81.03030395507812
571/643 Data:0.012 | Batch:0.583 | Total:0:03:45 | ETA:0:00:38 | Loss:0.40759365137947 | top1:81.040283203125
581/643 Data:0.002 | Batch:0.433 | Total:0:03:50 | ETA:0:00:35 | Loss:0.4076353173042534 | top1:81.03958129882812
591/643 Data:0.001 | Batch:0.611 | Total:0:03:55 | ETA:0:00:25 | Loss:0.40741884093195857 | top1:81.05245208740234
601/643 Data:0.001 | Batch:0.648 | Total:0:04:01 | ETA:0:00:26 | Loss:0.40742200890714036 | top1:81.06156158447266
611/643 Data:0.001 | Batch:0.716 | Total:0:04:08 | ETA:0:00:22 | Loss:0.4075226261358214 | to

591/643 Data:0.001 | Batch:0.548 | Total:0:05:13 | ETA:0:00:22 | Loss:0.40732756125947145 | top1:81.01607513427734
601/643 Data:0.001 | Batch:0.508 | Total:0:05:18 | ETA:0:00:23 | Loss:0.40763567177309173 | top1:80.99500274658203
611/643 Data:0.000 | Batch:0.350 | Total:0:05:22 | ETA:0:00:13 | Loss:0.4074464339866576 | top1:81.02045440673828
621/643 Data:0.001 | Batch:0.337 | Total:0:05:26 | ETA:0:00:09 | Loss:0.4074744314867133 | top1:81.03060150146484
631/643 Data:0.000 | Batch:0.340 | Total:0:05:29 | ETA:0:00:05 | Loss:0.4072850735270656 | top1:81.04753875732422
641/643 Data:0.000 | Batch:0.348 | Total:0:05:33 | ETA:0:00:01 | Loss:0.40714224284971 | top1:81.05460357666016
161/161 Data:0.000 | Batch:0.047 | Total:0:00:18 | ETA:0:00:00 | Loss:0.16485196686236658 | top1:99.82866668701172

Epoch: [25 | 300] LR: 0.079630
1/643 Data:1.466 | Batch:1.922 | Total:0:00:01 | ETA:0:20:35 | Loss:0.36528587341308594 | top1:84.5
11/643 Data:0.000 | Batch:0.328 | Total:0:00:05 | ETA:0:05:30 | Loss:

161/161 Data:0.000 | Batch:0.147 | Total:0:00:26 | ETA:0:00:00 | Loss:0.173468502093327 | top1:99.86293029785156

Epoch: [26 | 300] LR: 0.079571
1/643 Data:1.670 | Batch:2.146 | Total:0:00:02 | ETA:0:22:58 | Loss:0.4466801583766937 | top1:78.0
11/643 Data:0.001 | Batch:0.339 | Total:0:00:06 | ETA:0:06:03 | Loss:0.3998314440250397 | top1:81.63636779785156
21/643 Data:0.001 | Batch:0.409 | Total:0:00:09 | ETA:0:04:06 | Loss:0.39876380278950646 | top1:81.47618865966797
31/643 Data:0.001 | Batch:0.473 | Total:0:00:14 | ETA:0:04:33 | Loss:0.39864897247283687 | top1:81.51612854003906
41/643 Data:0.001 | Batch:0.418 | Total:0:00:18 | ETA:0:04:11 | Loss:0.3998747213584621 | top1:81.46341705322266
51/643 Data:0.001 | Batch:0.427 | Total:0:00:22 | ETA:0:03:47 | Loss:0.3972380622929218 | top1:81.76470184326172
61/643 Data:0.001 | Batch:0.430 | Total:0:00:26 | ETA:0:04:18 | Loss:0.39631720742241283 | top1:81.81967163085938
71/643 Data:0.001 | Batch:0.471 | Total:0:00:31 | ETA:0:04:20 | Loss:0.3973

51/643 Data:0.001 | Batch:0.592 | Total:0:00:21 | ETA:0:04:13 | Loss:0.4056004472807342 | top1:81.00980377197266
61/643 Data:0.001 | Batch:0.485 | Total:0:00:26 | ETA:0:04:25 | Loss:0.4042232168502495 | top1:81.11475372314453
71/643 Data:0.001 | Batch:0.398 | Total:0:00:31 | ETA:0:04:44 | Loss:0.4046923085837297 | top1:81.06338500976562
81/643 Data:0.001 | Batch:0.350 | Total:0:00:35 | ETA:0:04:00 | Loss:0.404901138794275 | top1:81.08024597167969
91/643 Data:0.009 | Batch:0.405 | Total:0:00:40 | ETA:0:04:09 | Loss:0.40397837725314467 | top1:81.19780731201172
101/643 Data:0.000 | Batch:0.521 | Total:0:00:45 | ETA:0:04:30 | Loss:0.4053131406850154 | top1:81.09900665283203
111/643 Data:0.000 | Batch:0.381 | Total:0:00:49 | ETA:0:04:13 | Loss:0.40431187308586397 | top1:81.16667175292969
121/643 Data:0.001 | Batch:0.447 | Total:0:00:55 | ETA:0:04:34 | Loss:0.40545314131689464 | top1:81.03718566894531
131/643 Data:0.002 | Batch:0.485 | Total:0:01:00 | ETA:0:05:00 | Loss:0.4049142872104208 | 